In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

llm = ChatOpenAI(
  temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

# 캐시가 저장되어 있는지 확인
# 저장되어 있으면 캐시에서 가져오고, 저장되어있지 않으면 캐시로 저장함
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

def load_memory(input):
  return memory.load_memory_variables({})["history"]

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
  MessagesPlaceholder(variable_name='history'),
  ("human", "{question}")
])

chain =  {"context": retriever, "question": RunnablePassthrough()} | RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
  result = chain.invoke(question)
  memory.save_context({"input": question}, {"output": result.content})
  print(result)

In [25]:
invoke_chain('Aaronson은 유죄인가요?')

content='제가 알기로는 Aaronson이 유죄인지 아닌지에 대한 정보는 제공되지 않았습니다.'


In [26]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Aaronson은 유죄인가요?'),
  AIMessage(content='제가 알기로는 Aaronson이 유죄인지 아닌지에 대한 정보는 제공되지 않았습니다.')]}

In [27]:
invoke_chain('그가 테이블에 어떤 메시지를 썼나요?')

content='그가 쓴 메시지는 다음과 같습니다:\nFREEDOM IS SLAVERY\nTWO AND TWO MAKE FIVE\nGOD IS POWER'


In [28]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Aaronson은 유죄인가요?'),
  AIMessage(content='제가 알기로는 Aaronson이 유죄인지 아닌지에 대한 정보는 제공되지 않았습니다.'),
  HumanMessage(content='그가 테이블에 어떤 메시지를 썼나요?'),
  AIMessage(content='그가 쓴 메시지는 다음과 같습니다:\nFREEDOM IS SLAVERY\nTWO AND TWO MAKE FIVE\nGOD IS POWER')]}

In [29]:
invoke_chain('Julia는 누구인가요?')

content='Julia는 주인공과 함께 이야기하는 여성 캐릭터입니다.'


In [30]:
invoke_chain('제가 방금 한 질문이 뭐였죠?')

content='Julia는 누구인가요?'
